In [0]:
%run "/Workspace/Users/sundaramloganathan@outlook.com/includes/storage_acc_config"

In [0]:
%run "../includes/folder_path"

In [0]:
dbutils.widgets.text("p_owner_name","")
v_owner_name = dbutils.widgets.get("p_owner_name")

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

results_raw_df = StructType([StructField("resultId", IntegerType(), True),
                         StructField("raceId", IntegerType(), True),
                         StructField("driverId", IntegerType(), True),
                         StructField("constructorId", IntegerType(), True),
                         StructField("number", IntegerType(), True),
                         StructField("grid", IntegerType(), True),
                         StructField("position", IntegerType(), True),
                         StructField("positionText", StringType(), True),
                         StructField("positionOrder", IntegerType(), True),
                         StructField("points", FloatType(), True),
                         StructField("laps", IntegerType(), True),
                         StructField("time", StringType(), True),
                         StructField("milliseconds", IntegerType(), True),
                         StructField("fastestLap", IntegerType(), True),
                         StructField("rank", IntegerType(), True),
                         StructField("fastestLapTime", StringType(), True),
                         StructField("fastestLapSpeed", StringType(), True),
                         StructField("statusId", StringType(), True)])

In [0]:
result_df = spark.read.options(header = True).schema(results_raw_df).json(f"{bronze_folder_path}/results.json")

In [0]:
from pyspark.sql.functions import current_timestamp, lit

result_updated_df = result_df.withColumnRenamed("resultId", "result_id") \
                            .withColumnRenamed("raceId", "race_id") \
                            .withColumnRenamed("driverId", "driver_id") \
                            .withColumnRenamed("constructorId", "constructor_id") \
                            .withColumnRenamed("positionText", "position_text") \
                            .withColumnRenamed("positionOrder", "position_order") \
                            .withColumnRenamed("fastestLap", "fastest_lap") \
                            .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                            .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                            .withColumn("ingested_by",lit(v_owner_name)) \
                            .withColumn("ingestion_date", current_timestamp()) \
                            .drop("statusId")

In [0]:
result_updated_df.write.partitionBy("race_id").mode("overwrite").parquet(f"{silver_folder_path}/results")

In [0]:
dbutils.notebook.exit("Success")